In [1]:
import pandas as pd
from datetime import timedelta
import ast

In [2]:
movies_df = pd.read_csv('imdb_movies.csv')

In [3]:
movies_df.head()

,title,duration,imdb_rating,directors,writers,cast,genres,release_date,languages,production_companies
0,Kabir's Path,PT57M,NaN,[],['Indubhushan Mandal'],[],['Documentary'],"January 1, 2024 (India)",['Hindi'],['Cinema Gang']
1,Tapori Nayak,NaN,8.7,[],[],"['Caroline Cave', 'Neil Grayston', 'Nitish Nir...",['Adventure'],"January 4, 2024 (India)",['Hindi'],[]
2,Rest in Manhole,PT1H25M,NaN,"['Joninder Singh', 'Vaibhav Soni']",[],['Bezwada Wilson'],['Drama'],"January 4, 2024 (India)",['Hindi'],[]
3,Jinn Chapter 1,PT46M,7.8,['Hamid Baig'],"['Hamid Baig', 'Fahad Khan']","['Naseeb Akhtar', 'Ayesha Athar', 'Mohammed Hu...",['Horror'],"July 11, 2024 (India)",['Hindi'],['Currant Noir Studios']
4,Tauba Tera Jalwa,PT2H10M,6.2,['Akashaditya Lama'],['Akashaditya Lama'],"['Ehsan Khan', 'Jahangir Khan', 'Jatin Khurana...","['Romance', 'Thriller']","January 5, 2024 (India)",['Hindi'],['Altair Media']


In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 370 non-null    object 
 1   duration              258 non-null    object 
 2   imdb_rating           215 non-null    float64
 3   directors             370 non-null    object 
 4   writers               370 non-null    object 
 5   cast                  370 non-null    object 
 6   genres                370 non-null    object 
 7   release_date          356 non-null    object 
 8   languages             370 non-null    object 
 9   production_companies  370 non-null    object 
dtypes: float64(1), object(9)
memory usage: 29.0+ KB


In [5]:
movies_df.describe(include='all')

,title,duration,imdb_rating,directors,writers,cast,genres,release_date,languages,production_companies
count,370,258,215.000000,370,370,370,370,356,370,370
unique,370,101,NaN,327,329,351,103,170,18,289
top,Dil Kya Kahe,PT2H,NaN,[],[],[],['Drama'],2024 (India),['Hindi'],[]
freq,1,12,NaN,26,35,20,102,19,345,70
mean,NaN,NaN,7.049767,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,1.558002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,3.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,8.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
movies_with_no_duration = movies_df[movies_df['duration'].isnull()]
movies_with_no_duration.to_csv('movies_with_no_duration.csv', index=False)
movies_df = movies_df.dropna(subset=['duration'])

In [7]:
movies_with_no_rating = movies_df[movies_df['imdb_rating'].isnull()]
movies_with_no_rating.to_csv('movies_with_no_rating.csv', index=False)
movies_df = movies_df.dropna(subset=['imdb_rating'])

In [8]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 181 entries, 3 to 330
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 181 non-null    object 
 1   duration              181 non-null    object 
 2   imdb_rating           181 non-null    float64
 3   directors             181 non-null    object 
 4   writers               181 non-null    object 
 5   cast                  181 non-null    object 
 6   genres                181 non-null    object 
 7   release_date          181 non-null    object 
 8   languages             181 non-null    object 
 9   production_companies  181 non-null    object 
dtypes: float64(1), object(9)
memory usage: 15.6+ KB


In [9]:
movies_df.describe(include='all')

,title,duration,imdb_rating,directors,writers,cast,genres,release_date,languages,production_companies
count,181,181,181.000000,181,181,181,181,181,181,181
unique,181,84,NaN,170,171,178,82,91,12,160
top,Jinn Chapter 1,PT2H2M,NaN,[],[],[],['Drama'],"July 26, 2024 (India)",['Hindi'],[]
freq,1,8,NaN,4,10,4,43,7,164,15
mean,NaN,NaN,6.796685,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,1.469539,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,3.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,5.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,6.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,7.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
movies_with_more_than_8_rating = movies_df[movies_df['imdb_rating'] > 8]
movies_with_more_than_8_rating.to_csv('movies_with_more_than_8_rating.csv', index=False)
movies_df = movies_df[movies_df['imdb_rating'] <= 8]

In [11]:
movies_df.to_csv('cleaned_imdb_movies.csv', index=False)
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 3 to 330
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 143 non-null    object 
 1   duration              143 non-null    object 
 2   imdb_rating           143 non-null    float64
 3   directors             143 non-null    object 
 4   writers               143 non-null    object 
 5   cast                  143 non-null    object 
 6   genres                143 non-null    object 
 7   release_date          143 non-null    object 
 8   languages             143 non-null    object 
 9   production_companies  143 non-null    object 
dtypes: float64(1), object(9)
memory usage: 12.3+ KB


In [12]:
movies_df.describe(include='all')

,title,duration,imdb_rating,directors,writers,cast,genres,release_date,languages,production_companies
count,143,143,143.000000,143,143,143,143,143,143,143
unique,143,75,NaN,134,136,140,71,76,9,130
top,Jinn Chapter 1,PT2H2M,NaN,[],[],[],['Drama'],"May 31, 2024 (India)",['Hindi'],[]
freq,1,8,NaN,4,7,4,30,5,130,9
mean,NaN,NaN,6.255944,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,1.126294,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,3.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,5.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,6.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,7.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
movies_df.head()

,title,duration,imdb_rating,directors,writers,cast,genres,release_date,languages,production_companies
3,Jinn Chapter 1,PT46M,7.8,['Hamid Baig'],"['Hamid Baig', 'Fahad Khan']","['Naseeb Akhtar', 'Ayesha Athar', 'Mohammed Hu...",['Horror'],"July 11, 2024 (India)",['Hindi'],['Currant Noir Studios']
4,Tauba Tera Jalwa,PT2H10M,6.2,['Akashaditya Lama'],['Akashaditya Lama'],"['Ehsan Khan', 'Jahangir Khan', 'Jatin Khurana...","['Romance', 'Thriller']","January 5, 2024 (India)",['Hindi'],['Altair Media']
5,Fire of Love: RED,PT2H10M,3.8,['Ashok Tyagi'],['Ashok Tyagi'],"['Bharat Dabholkar', 'Krushna Abhishek', 'Shan...",['Thriller'],"January 5, 2024 (India)",['Hindi'],['Avanti Prajakta Arts']
6,Saar the Conclusion,PT1H31M,7.9,['Danish Ali'],"['Kamleshwar', 'Sanjay Kumar']","['Kapil Sharma', 'Monika Choudhary', 'Rupali P...",['Drama'],"January 5, 2024 (India)",['Hindi'],"['Kavyanjali Films', 'Mishika Films']"
11,Merry Christmas,PT2H24M,6.9,['Sriram Raghavan'],"['Frédéric Dard', 'Sriram Raghavan', 'Arijit B...","['Katrina Kaif', 'Vijay Sethupathi', 'Tinnu An...","['Drama', 'Mystery', 'Thriller']","January 12, 2024 (India)","['Hindi', 'Tamil']","['Matchbox Pictures', 'Tips Films']"


In [14]:
movies_df['release_date'] = movies_df['release_date'].apply(lambda x: x.split('(')[0].strip().strip())
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], format='mixed')

In [15]:
movies_df['duration'] = movies_df['duration'].replace('PT', '', regex=True)

In [16]:
def str_to_time(duration):
    hours = int(duration.split('H')[0]) if 'H' in duration else 0
    minutes = int(duration.split('M')[0].split('H')[-1]) if 'M' in duration else 0
    return f"{hours:02}:{minutes:02}:00"

In [17]:
movies_df['duration'] = movies_df['duration'].apply(str_to_time)
movies_df['duration'] = pd.to_datetime(movies_df['duration'], format='%H:%M:%S').dt.time

In [18]:
movies_df.head()

,title,duration,imdb_rating,directors,writers,cast,genres,release_date,languages,production_companies
3,Jinn Chapter 1,00:46:00,7.8,['Hamid Baig'],"['Hamid Baig', 'Fahad Khan']","['Naseeb Akhtar', 'Ayesha Athar', 'Mohammed Hu...",['Horror'],2024-07-11,['Hindi'],['Currant Noir Studios']
4,Tauba Tera Jalwa,02:10:00,6.2,['Akashaditya Lama'],['Akashaditya Lama'],"['Ehsan Khan', 'Jahangir Khan', 'Jatin Khurana...","['Romance', 'Thriller']",2024-01-05,['Hindi'],['Altair Media']
5,Fire of Love: RED,02:10:00,3.8,['Ashok Tyagi'],['Ashok Tyagi'],"['Bharat Dabholkar', 'Krushna Abhishek', 'Shan...",['Thriller'],2024-01-05,['Hindi'],['Avanti Prajakta Arts']
6,Saar the Conclusion,01:31:00,7.9,['Danish Ali'],"['Kamleshwar', 'Sanjay Kumar']","['Kapil Sharma', 'Monika Choudhary', 'Rupali P...",['Drama'],2024-01-05,['Hindi'],"['Kavyanjali Films', 'Mishika Films']"
11,Merry Christmas,02:24:00,6.9,['Sriram Raghavan'],"['Frédéric Dard', 'Sriram Raghavan', 'Arijit B...","['Katrina Kaif', 'Vijay Sethupathi', 'Tinnu An...","['Drama', 'Mystery', 'Thriller']",2024-01-12,"['Hindi', 'Tamil']","['Matchbox Pictures', 'Tips Films']"


In [19]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 3 to 330
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   title                 143 non-null    object        
 1   duration              143 non-null    object        
 2   imdb_rating           143 non-null    float64       
 3   directors             143 non-null    object        
 4   writers               143 non-null    object        
 5   cast                  143 non-null    object        
 6   genres                143 non-null    object        
 7   release_date          143 non-null    datetime64[ns]
 8   languages             143 non-null    object        
 9   production_companies  143 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 12.3+ KB


In [20]:
movies_df.describe(include='all')

,title,duration,imdb_rating,directors,writers,cast,genres,release_date,languages,production_companies
count,143,143,143.000000,143,143,143,143,143,143,143
unique,143,75,NaN,134,136,140,71,NaN,9,130
top,Jinn Chapter 1,02:02:00,NaN,[],[],[],['Drama'],NaN,['Hindi'],[]
freq,1,8,NaN,4,7,4,30,NaN,130,9
mean,NaN,NaN,6.255944,NaN,NaN,NaN,NaN,2024-06-26 14:15:56.643356672,NaN,NaN
min,NaN,NaN,3.400000,NaN,NaN,NaN,NaN,2024-01-01 00:00:00,NaN,NaN
25%,NaN,NaN,5.500000,NaN,NaN,NaN,NaN,2024-03-29 00:00:00,NaN,NaN
50%,NaN,NaN,6.400000,NaN,NaN,NaN,NaN,2024-06-28 00:00:00,NaN,NaN
75%,NaN,NaN,7.100000,NaN,NaN,NaN,NaN,2024-09-20 00:00:00,NaN,NaN
max,NaN,NaN,8.000000,NaN,NaN,NaN,NaN,2024-12-25 00:00:00,NaN,NaN


In [21]:
actors = list(pd.read_csv('Actors.csv')['Actors'])
actresses = list(pd.read_csv('Actresses.csv')['Actresses'])

In [22]:
movies_df['cast'] = movies_df['cast'].apply(ast.literal_eval)
movies_df["actor"] = movies_df["cast"].apply(lambda x: [name for name in x if name in actors])
movies_df["actress"] = movies_df["cast"].apply(lambda x: [name for name in x if name in actresses])

In [23]:
movies_df.drop(columns=['cast'], inplace=True)

In [24]:
movies_df

,title,duration,imdb_rating,directors,writers,genres,release_date,languages,production_companies,actor,actress
3,Jinn Chapter 1,00:46:00,7.8,['Hamid Baig'],"['Hamid Baig', 'Fahad Khan']",['Horror'],2024-07-11,['Hindi'],['Currant Noir Studios'],"[Naseeb Akhtar, Mohammed Humza Baig, Rahat Hus...","[Ayesha Athar, Manjari Chaudhary]"
4,Tauba Tera Jalwa,02:10:00,6.2,['Akashaditya Lama'],['Akashaditya Lama'],"['Romance', 'Thriller']",2024-01-05,['Hindi'],['Altair Media'],"[Ehsan Khan, Jahangir Khan, Jatin Khurana, Sha...","[Angela Krislinzki, Ameesha Patel]"
5,Fire of Love: RED,02:10:00,3.8,['Ashok Tyagi'],['Ashok Tyagi'],['Thriller'],2024-01-05,['Hindi'],['Avanti Prajakta Arts'],"[Bharat Dabholkar, Krushna Abhishek, Shantanu ...","[Payal Ghosh, Kanchan Bhor, Simran Ahuja, Neel..."
6,Saar the Conclusion,01:31:00,7.9,['Danish Ali'],"['Kamleshwar', 'Sanjay Kumar']",['Drama'],2024-01-05,['Hindi'],"['Kavyanjali Films', 'Mishika Films']","[Kapil Sharma, Kamleshwar, Manoj Chauhan, Abhi...","[Monika Choudhary, Rupali Pallavi, Priyanshi S..."
11,Merry Christmas,02:24:00,6.9,['Sriram Raghavan'],"['Frédéric Dard', 'Sriram Raghavan', 'Arijit B...","['Drama', 'Mystery', 'Thriller']",2024-01-12,"['Hindi', 'Tamil']","['Matchbox Pictures', 'Tips Films']","[Vijay Sethupathi, Vinay Pathak, Sanjay Kapoor...","[Katrina Kaif, Pari Sharma, Gayathrie, Ashwini..."
...,...,...,...,...,...,...,...,...,...,...,...
320,Benaam Rishta,01:30:00,6.4,['Sanjay Verma'],['Giselle Galbao'],['Romance'],2024-12-11,['Hindi'],['Canopy'],"[Suraj Pratap Singh, Suraj Sonik]",[Rose Khan]
327,Vanvaas,02:40:00,6.7,['Anil Sharma'],"['Amjad Ali', 'Anil Sharma', 'Sunil Sirvaiya']","['Action', 'Drama']",2024-12-20,['Hindi'],"['Anil Sharma Productions', 'Zee Studios']","[Nana Patekar, Rajpal Naurang Yadav, Kettan Si...","[Ashwini Kalsekar, Shruti Marathe, Simratt Kau..."
328,Yo Yo Honey Singh: Famous,01:20:00,6.9,['Mozez Singh'],[],"['Documentary', 'Drama']",2024-12-20,['Hindi'],['Sikhya Entertainment'],"[Shah Rukh Khan, Salman Khan, Yo Yo Honey Sing...",[Jazzy B]
329,Baby John,02:39:00,6.4,['Kalees'],"['Sumit Arora', 'Kalees']","['Action', 'Drama', 'Thriller']",2024-12-25,['Hindi'],"['A for Apple Productions', 'Cine1 Studios', '...","[Salman Khan, Varun Dhawan, Jackie Shroff, Rit...","[Wamiqa Gabbi, Keerthy Suresh, Sanya Malhotra,..."


In [25]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 3 to 330
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   title                 143 non-null    object        
 1   duration              143 non-null    object        
 2   imdb_rating           143 non-null    float64       
 3   directors             143 non-null    object        
 4   writers               143 non-null    object        
 5   genres                143 non-null    object        
 6   release_date          143 non-null    datetime64[ns]
 7   languages             143 non-null    object        
 8   production_companies  143 non-null    object        
 9   actor                 143 non-null    object        
 10  actress               143 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 13.4+ KB


In [26]:
movies_df.describe(include='all')

,title,duration,imdb_rating,directors,writers,genres,release_date,languages,production_companies,actor,actress
count,143,143,143.000000,143,143,143,143,143,143,143,143
unique,143,75,NaN,134,136,71,NaN,9,130,139,135
top,Jinn Chapter 1,02:02:00,NaN,[],[],['Drama'],NaN,['Hindi'],[],[],[]
freq,1,8,NaN,4,7,30,NaN,130,9,5,9
mean,NaN,NaN,6.255944,NaN,NaN,NaN,2024-06-26 14:15:56.643356672,NaN,NaN,NaN,NaN
min,NaN,NaN,3.400000,NaN,NaN,NaN,2024-01-01 00:00:00,NaN,NaN,NaN,NaN
25%,NaN,NaN,5.500000,NaN,NaN,NaN,2024-03-29 00:00:00,NaN,NaN,NaN,NaN
50%,NaN,NaN,6.400000,NaN,NaN,NaN,2024-06-28 00:00:00,NaN,NaN,NaN,NaN
75%,NaN,NaN,7.100000,NaN,NaN,NaN,2024-09-20 00:00:00,NaN,NaN,NaN,NaN
max,NaN,NaN,8.000000,NaN,NaN,NaN,2024-12-25 00:00:00,NaN,NaN,NaN,NaN


In [27]:
movies_df['directors'] = movies_df['directors'].apply(lambda x: ast.literal_eval(x))
movies_df['writers'] = movies_df['writers'].apply(lambda x: ast.literal_eval(x))
movies_df['genres'] = movies_df['genres'].apply(lambda x: ast.literal_eval(x))
movies_df['languages'] = movies_df['languages'].apply(lambda x: ast.literal_eval(x))
movies_df['production_companies'] = movies_df['production_companies'].apply(lambda x: ast.literal_eval(x))

In [28]:
movies_df['directors'].explode().value_counts()

directors
Akashaditya Lama     2
Pulkit               2
Anees Bazmee         2
Aditya Sarpotdar     2
Shoib Nikash Shah    2
                    ..
Sanjay Verma         1
Anil Sharma          1
Mozez Singh          1
Kalees               1
Ratnaa Sinha         1
Name: count, Length: 145, dtype: int64

In [29]:
movies_df['writers'].explode().value_counts()

writers
Abbas Dalal          4
Hussain Dalal        3
Pulkit               2
Akashaditya Lama     2
Suprotim Sengupta    2
                    ..
Giselle Galbao       1
Amjad Ali            1
Anil Sharma          1
Sunil Sirvaiya       1
Kalees               1
Name: count, Length: 252, dtype: int64

In [30]:
movies_df['genres'].explode().value_counts()

genres
Drama                      96
Thriller                   37
Action                     31
Comedy                     28
Romance                    22
Crime                      21
Biography                   9
Mystery                     9
Horror                      8
History                     6
Sport                       6
Family                      5
Docudrama                   4
Adventure                   4
Animation                   3
Documentary                 3
Action Epic                 2
Whodunnit                   2
Legal Drama                 2
Supernatural Horror         2
Psychological Drama         2
Romantic Comedy             2
Fantasy                     2
Period Drama                2
Sci-Fi                      2
Political Drama             1
One-Person Army Action      1
Psychological Horror        1
Psychological Thriller      1
Political Thriller          1
Extreme Sport               1
Globetrotting Adventure     1
Soccer                      1
Art

In [31]:
movies_df['languages'].explode().value_counts()

languages
Hindi        143
English        5
Marathi        3
Tamil          1
Telugu         1
Gujarati       1
Punjabi        1
Ladakhi        1
Malayalam      1
Name: count, dtype: int64

In [32]:
movies_df['production_companies'].explode().value_counts()

production_companies
Jio Studios                11
T-Series Films              8
Zee Studios                 6
Maddock Films               5
Dharma Productions          4
                           ..
Canopy                      1
Anil Sharma Productions     1
A for Apple Productions     1
Cine1 Studios               1
Trizent Mediaworks          1
Name: count, Length: 201, dtype: int64

In [33]:
movies_df['actor'].explode().value_counts()

actor
Rajesh Sharma           6
Rajpal Naurang Yadav    6
Ajay Devgn              5
Anil Rastogi            5
Vijay Raaz              5
                       ..
Manikandan K.           1
Sonalli Sharmisstha     1
Jaffer Sadiq            1
Armaan Khera            1
Jahangir Khan           1
Name: count, Length: 998, dtype: int64

In [34]:
movies_df['actress'].explode().value_counts()

actress
Ashwini Kalsekar         5
Sheeba Chaddha           4
Maahi Jain               3
Chhaya Kadam             3
Kriti Sanon              3
                        ..
Seerat Mast              1
Aditi Sandhya Sharma     1
Banveen Singh            1
Kathy Blaze Jefferson    1
Ameesha Patel            1
Name: count, Length: 510, dtype: int64